In [1]:
%load_ext autoreload
%autoreload 2

# Data Preprocessing

Convert STL to TXT files and their metadata

In [2]:
STL_ROOT = 'data/Foot_Last_stl/STL'

In [2]:
from data_utils import helpers
from pathlib import Path
import re
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
stl_files_list = [] # input
txt_files_list = [] # output

for root, dirs, files in os.walk(STL_ROOT, topdown=False):
    txt_root = re.sub('(?i)stl', 'txt', root)
    Path(txt_root).mkdir(parents=True, exist_ok=True)
    stl_files_list += [os.path.join(root, file) for file in files]

txt_files_list = list(map(lambda stl_file: re.sub('(?i)stl', 'txt', stl_file), stl_files_list))

In [5]:
# TEST_FILE = "data/Foot_Last_stl/STL/Foot_stl/Foot_Right_stl/202212071755ra.stl
# helpers.stl_to_xyz_with_normals_vectorized(TEST_FILE, 'temp/c.txt', stride=10)

In [28]:
# Rename
DATA_ROOT = '/home/hoang/Projects/Pointnet_Pointnet2_pytorch/data/modelnet40_withFootLast_AllLeft/'
subdirs = ['foot', 'last']
EXT = '.txt'
DOWNSAMPLING = 10

import os
import glob
from tqdm import tqdm

# # CONVERT stl to txt and (optionally) downsample
# for stl, txt in tqdm(zip(stl_files_list, txt_files_list), total=len(txt_files_list)):
#     helpers.stl_to_xyz_with_normals_vectorized(stl, txt, stride=DOWNSAMPLING)

# Rename for easy process
# for subdir in subdirs:
#     for idx, fn in enumerate(glob.glob(os.path.join(DATA_ROOT, subdir, f'*{EXT}'))):
#         os.rename(fn, os.path.join(DATA_ROOT, subdir, f'{subdir}_{idx}{EXT}'))


Now, please move move and symbolically link files into a new directory so we can make use of provided ModelNetDataset class by the authors. After, we'll create some text files containing labels and files list for train / test.

In [6]:
import glob 
import random

DATA_ROOT = '/home/hoang/Projects/Pointnet_Pointnet2_pytorch/data/modelnet40_withFootLast/'
TRAIN_SPLIT = .8
SEED = 42
EXT = '.txt'

files_list = []
train_files = []
test_files = []

random.seed(SEED)
for root, subdirs, _ in os.walk(DATA_ROOT):
    subdirs =[ sd for sd in subdirs if not sd.startswith('.')]
    for sdir in subdirs:
        all_files = [fn.replace(DATA_ROOT, '') for fn in glob.glob(os.path.join(DATA_ROOT, sdir, "*.txt"))]
        files_list += all_files
        all_files = [os.path.splitext(os.path.basename(file))[0] for file in all_files]
        random.shuffle(all_files)
        n_train = int(TRAIN_SPLIT*len(all_files))
        train_files += all_files[:n_train]
        test_files += all_files[n_train:]

    if len(subdirs) > 0:
        # write all shape classes
        with open(os.path.join(DATA_ROOT, 'modelnet40_shape_names.txt'), 'w') as ff:
            ff.writelines("\n".join(subdirs))

# write all shape files
with open(os.path.join(DATA_ROOT, 'filelist.txt'), 'w') as ff:
    ff.writelines("\n".join(set(files_list)))



# write training files. I used same names as modelnet40 to make use of prewritten classes easily
with open(os.path.join(DATA_ROOT, 'modelnet40_train.txt'), 'w') as ff:
    ff.writelines("\n".join(set(train_files)))

# write test files
with open(os.path.join(DATA_ROOT, 'modelnet40_test.txt'), 'w') as ff:
    ff.writelines("\n".join(set(test_files)))



## Dataset class
